In [100]:
from sklearn import preprocessing
import PIL.Image as pilimg
import matplotlib.pyplot as plt
import csv
import numpy as np
from numpy import *
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from models.densenet import densenet121
from models.vgg import VGGNet

%matplotlib inline 

# Data Preprocessing

In [2]:
class C100Dataset:
    """
    X is a feature vector
    Y is the predictor variable
    """

    def __init__(self, filename):
        ## read the csv for dataset (cifar100.csv, cifar100_lt.csv or cifar100_nl.csv), 
        # 
        # Format:
        #   image file path,classname
        
        ### TODO: Read the csv file and make the training and testing set
        ## YOUR CODE HERE
        self.filename=filename
        f=open(filename)
        rdr=csv.reader(f)
        path=[]
        label=[]
        
        for line in rdr:
            rpath=line[0].replace('cifar100','../dataset')
            path.append(rpath)
            label.append(line[1])
        f.close()

        self.tr_x = []
        self.tr_y = []
        self.ts_x = []
        self.ts_y = []


        for i in range(len(label)):
            img = pilimg.open(path[i])#path 읽어와서 이미지 열기
            pixel = np.array(img)
            pixel = np.transpose(pixel, (2, 1, 0))
            cat=path[i].split('/')
            if cat[3]=='train':
                self.tr_x.append(pixel)
                self.tr_y.append(label[i])
            else:
                self.ts_x.append(pixel)
                self.ts_y.append(label[i])
                
    def getDataset(self):
        self.tr_x = np.array(self.tr_x)
        self.ts_x = np.array(self.ts_x)
        
        le = preprocessing.LabelEncoder()
        le.fit(self.tr_y)
        self.tr_y = le.transform(self.tr_y)
        
        le.fit(self.ts_y)
        self.ts_y = le.transform(self.ts_y)
        
        return [self.tr_x, self.tr_y, self.ts_x, self.ts_y]

In [91]:
dataset_odn = C100Dataset('../dataset/data/cifar100.csv')
[data_odn_tr_x, data_odn_tr_y, data_odn_ts_x, data_odn_ts_y] = dataset_odn.getDataset()

In [92]:
data_odn_tr_x = torch.Tensor(data_odn_tr_x)
data_odn_ts_x = torch.Tensor(data_odn_ts_x)

In [93]:
type(data_odn_tr_y)

numpy.ndarray

In [94]:
train_dataset = []
for i, (tr_y) in enumerate(data_odn_tr_y):
    train_dataset.append((data_odn_tr_x[i],int(tr_y)))
    
test_dataset = []
for i, (ts_y) in enumerate(data_odn_ts_y):
    test_dataset.append((data_odn_ts_x[i],int(ts_y)))

In [95]:
len(train_dataset)

50000

In [96]:
len(test_dataset)

10000

In [109]:
# device setting
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

# Hyper parameters
num_epochs = 30 # number of epochs for train
batch_size = 32 # do not change this value 128
learning_rate = 0.0001 # do not change this value
num_classes = 100

cuda:0


In [110]:
# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)